# 🌐 Web Auth Signing을 사용한 AgentCore Browser 

## 개요

이 튜토리얼에서는 Amazon Bedrock AgentCore Browser tool에서 Web Bot Auth signing을 활성화하는 방법을 배웁니다. 

### 튜토리얼 세부사항

| 정보                | 세부사항                                                                          |
|:--------------------|:---------------------------------------------------------------------------------|
| 튜토리얼 유형       | 대화형                                                                            |
| Agent 유형          | 단일                                                                              |
| Agentic Framework   | Strands                                                                          |
| LLM model           | Claude 4.5 Haiku                                                                 |
| 튜토리얼 구성요소   | Browser 자동화, Browser 요청의 *Web Bot Auth* signing                            |
| 튜토리얼 분야       |                                                                                  |
| 예제 복잡도         | 중급                                                                              |
| 사용된 SDK          | Amazon Bedrock AgentCore Python SDK, Strands Agents, Strands Agent Tools         |
 
### 튜토리얼 아키텍처

![Architecture](images/Architecture.png)

### 튜토리얼 주요 기능

* Strands agent와 함께 headless 방식으로 browser tool 사용
* strands_tools AgentCoreBrowser tool 사용
* 빠르고 효율적인 분석을 위한 Claude 4.5 Haiku model

## 사전 요구사항

이 튜토리얼을 실행하려면 다음이 필요합니다:
* Python 3.10+
* 구성된 AWS 자격 증명
* Amazon Bedrock AgentCore SDK
* Strands agents 및 strands-agents-tools 패키지
* Amazon Bedrock의 Claude 4.5 Haiku model에 대한 액세스

**구현**: agent에서 browser 자동화를 위해 공식 `strands_tools.browser.AgentCoreBrowser`를 사용합니다.

## 🔧 AgentCore Browser 구성

AgentCore Browser tool을 사용하기 전에, 녹화 기능, 네트워크 구성, 실행 역할과 같은 특정 설정으로 커스텀 browser 구성을 선택적으로 생성할 수 있습니다. 이 섹션에서는 AWS SDK를 사용하여 커스텀 browser 구성을 생성하는 방법을 보여줍니다.

### 커스텀 Browser 구성 생성

다음 코드는 녹화가 활성화되고 특정 실행 역할이 있는 커스텀 AgentCore Browser를 생성하는 방법을 보여줍니다.

#### 📚 Web Bot Auth Signing을 사용한 Browser 구성 생성

**Browser Signing이란?**

Browser signing (`browserSigning.enabled = True`)은 AgentCore Browser가 모든 나가는 HTTP 요청에 자동으로 서명하도록 구성합니다. 이는 다음에 필수적입니다:

- **인증된 API 액세스**: 보호된 API에 대한 요청 서명
- **AWS 서비스 통합**: AWS 자격 증명으로 요청 자동 서명
- **보안 규정 준수**: 모든 browser 요청이 인증되도록 보장
- **요청 무결성**: 요청 헤더에 암호화 서명

활성화되면 browser는 다음을 수행합니다:
1. 모든 HTTP/HTTPS 요청 가로채기
2. 요청 헤더에 암호화 서명 추가
3. 인증 토큰 자동 포함
4. 요청 간 세션 무결성 유지


### 📦 라이브러리 가져오기


In [ ]:
import boto3
import uuid
import os
import sys
from strands import Agent
from strands_tools.browser import AgentCoreBrowser  # AgentCore Browser tool
import asyncio
import time

# 상위 디렉토리의 utils 모듈 접근을 위한 경로 설정
tutorials_path = os.path.abspath(os.path.join(os.getcwd(), '../../../'))
if tutorials_path not in sys.path:
    sys.path.insert(0, tutorials_path)

from utils import create_agentcore_role

# AgentCore 제어 평면 클라이언트 생성
cp_client = boto3.client('bedrock-agentcore-control', 
                         region_name='us-west-2')

# AWS 계정 정보 조회
accountId = boto3.client("sts").get_caller_identity()["Account"] 
region = boto3.Session().region_name

print(f"Account ID: {accountId}")
print(f"Region: {region}")

### 🔧 설정 및 역할 생성


In [ ]:
# AgentCore Browser 실행을 위한 IAM 역할 생성
execution_role_arn = create_agentcore_role("web-bot-auth")["Role"]["Arn"]

print(f"\n✅ Role Created Successfully : {execution_role_arn}")

# Web Bot Auth signing이 활성화된 커스텀 browser 인스턴스 생성
response = cp_client.create_browser(
    name="web_bot_auth_browser_" + str(uuid.uuid4())[:6],
    description="Browser configured to sign web bot auth",
    networkConfiguration={
        "networkMode": "PUBLIC"  # 공용 네트워크 모드 사용
    },
    executionRoleArn=execution_role_arn,
    browserSigning={
        "enabled": True  # Web Bot Auth signing 활성화 - 모든 HTTP 요청에 자동 서명
    }
)

browserId = response['browserId']
browserArn = response['browserArn']
print(f"\n✅ Browser Created Successfully!")
print(f"   Browser ID: {browserId}")
print(f"   Browser ARN: {browserArn}")
print(f"\n🔐 Browser signing is ENABLED - all requests will be automatically signed")

### 🔧 AgentCoreBrowser Tool을 사용한 Strands Agent 생성


In [ ]:
# Create and configure the Strands agent with AgentCoreBrowser
# signing이 활성화된 커스텀 browser로 AgentCoreBrowser tool 초기화
agent_core_browser = AgentCoreBrowser(identifier=browserId, region="us-west-2")
# signing이 비활성화된 기본 browser로 AgentCoreBrowser tool 초기화 (비교용)
agent_core_default_browser = AgentCoreBrowser(region="us-west-2")

# SequentialToolExecutor를 사용하여 browser 작업의 동시 실행 방지
from strands.tools.executors import SequentialToolExecutor
    
# signing이 활성화된 agent 생성 - Claude 4.5 Haiku 모델 사용
strands_agent = Agent(
    tools=[agent_core_browser.browser],  # signing이 활성화된 커스텀 browser 사용
    tool_executor=SequentialToolExecutor(),  # browser 작업을 순차적으로 실행하여 asyncio 충돌 방지
    model="global.anthropic.claude-haiku-4-5-20251001-v1:0",
    system_prompt="""You are a website analyst with browser signing capabilities.
1. Use the browser tool to visit and interact with the website EFFICIENTLY
2. Focus on extracting key information QUICKLY and within 2-3 browser interactions.
3. Review browser requests for signatures related to Web Bot Auth's Signature and Signature-Agent http headers, 
   to verify if browser signing is configured."""
)

# signing이 비활성화된 agent 생성 - 비교 테스트용
strands_agent_unsigned = Agent(
    tools=[agent_core_default_browser.browser],  # signing이 비활성화된 기본 browser 사용
    tool_executor=SequentialToolExecutor(),
    model="global.anthropic.claude-haiku-4-5-20251001-v1:0",
    system_prompt="""You are a website analyst with browser signing capabilities.
1. Use the browser tool to visit and interact with the website EFFICIENTLY
2. Focus on extracting key information QUICKLY and within 2-3 browser interactions.
3. Review browser requests for signatures related to Web Bot Auth's Signature and Signature-Agent http headers, 
   to verify if browser signing is configured."""
)

In [ ]:
# Define async function with sequential execution (no more concurrent conflicts)
async def analyze_website(agent, prompt):
    """Async wrapper for agent invocation with sequential tool execution"""
    try:
        # SequentialToolExecutor 사용으로 browser 작업 충돌 방지
        result = await agent.invoke_async(prompt)
        return result
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        import traceback
        traceback.print_exc()
        return None

### 🚀 Browser Signing이 활성화된 상태로 분석 시작


In [ ]:
print("\n🚀 Validate browser signing against CloudFlare's crawltest site.")
print("="*100)
  
# signing이 활성화된 agent로 CloudFlare의 Web Bot Auth 테스트 사이트 분석
result_signed = await analyze_website(
    strands_agent,
    "Review the output and status code at https://crawltest.com/cdn-cgi/web-bot-auth and provide 3 to 4 concise key insights, based on https://developers.cloudflare.com/bots/reference/bot-verification/web-bot-auth/"
)

if result_signed:
    print("\n\n✅ Analysis completed, with Web Bot Auth browser signing enabled")
    print("-"*100)
    print(result_signed)
    print("-"*100)

### browser signing 없이 실험 재실행

테스트가 유효한지 확인하기 위해, browser signing 없이 구성된 agent로 동일한 프롬프트를 실행해 보겠습니다.

In [ ]:
print("\n🚀 Validate browser signing against CloudFlare's crawltest site - without Web Bot Auth browser signing.")
print("="*100)
  
# signing이 비활성화된 agent로 동일한 테스트 실행 (비교용)
result_unsigned = await analyze_website(
    strands_agent_unsigned,
    "Review the output and status code at https://crawltest.com/cdn-cgi/web-bot-auth and provide 3 to 4 concise key insights, based on https://developers.cloudflare.com/bots/reference/bot-verification/web-bot-auth/"
)

if result_unsigned:
    print("\n\n✅ Analysis completed - without Web Bot Auth browser signing")
    print("-"*100)
    print(result_unsigned)
    print("-"*100)

결과를 비교해 보겠습니다:

In [ ]:
# Compare result_unsigned and result_signed using Strands Agent
# signing 활성화/비활성화 결과를 비교하는 프롬프트 생성
comparison_prompt = f"""
Please analyze and compare these two agent outputs side by side:

**Unsigned Agent Output:**
{result_unsigned}

**Signed Agent Output:**
{result_signed}

Please provide:
* A side-by-side comparison highlighting key differences
* Validation of the Signed Agent using Signatures correctly
* Validation of the Unsigned Agent NOT using Signatures
* Summary of which aspects differ most significantly

Format your response clearly with headers and bullet points for easy reading.
"""

# 결과 비교를 위한 별도의 agent 생성
comparison_agent = Agent(
    system_prompt="""You are an expert data analyst evaluating different AI agent outputs.""",
    callback_handler=None
)

comparison_response = comparison_agent(comparison_prompt)

print("=== COMPARISON OF AGENT OUTPUTS ===")
print(comparison_response)


## 🎭 내부 동작 원리

**browser signing이 활성화된** 상태로 이 노트북을 실행하면 다음 프로세스가 발생합니다:

### 1. **Browser 구성 생성**
```python
browserSigning={
    "enabled": True
}
```
이는 AgentCore Browser 서비스에 browser가 만드는 모든 HTTP 요청에 자동으로 서명하도록 지시합니다.

### 2. **Agent 초기화**
Strands agent는 다음으로 초기화됩니다:
- signing이 활성화된 browser 구성 식별자. *참고* 이것은 기본 browser 식별자가 아닙니다.
- Claude 4.5 Haiku model
- Browser tool 통합

### 3. **요청 서명 흐름**
agent가 웹사이트로 이동할 때:

```text
┌─────────────────┐    ┌───────────────────┐    ┌──────────────────────┐    ┌──────────────┐
│  Agent Request  │ ──→│ AgentCore Browser │ ──→│ Sign Request Headers │ ──→│Target Website│
└─────────────────┘    └───────────────────┘    └──────────────────────┘    └──────────────┘
                                                          │
                                                          ▼
                                                Add Web Bot Auth headers:
                                                • Signature-Input header
                                                • Signature-Agent header
                                                • Signature header

```

## 🔒 보안 이점

browser signing이 활성화되면:

✅ **자동 인증** - 수동 헤더 관리 불필요  
✅ **요청 무결성** - 암호화 서명으로 변조 방지  
✅ **AWS 통합** - agent가 browser tool을 사용하기 위한 원활한 IAM 자격 증명 통합 
✅ **세션 관리** - 각 세션이 자체 browser 세션을 갖는 안전한 세션 토큰 처리 
✅ **감사 추적** - 모든 서명된 요청 로깅 가능  

## 🛠️ 문제 해결

### 오류: "Browser session not found"
**해결책**: browser ID가 만료되었을 수 있습니다. browser 생성 셀(cell-3)을 다시 실행하세요

### RuntimeError: "Leaving task does not match the current task"
**해결책**: 이 asyncio 오류는 browser 작업이 동시에 실행될 때 발생합니다. 노트북은 `SequentialToolExecutor()`를 사용하여 browser 작업이 한 번에 하나씩 실행되도록 보장함으로써 이 문제를 방지합니다.

**발생 이유**: agent가 AgentCore Browser tool에 여러 비동기 작업(get_html, get_text, screenshot 등)을 병렬로 수행하도록 요청하면, Strands가 이를 동시에 실행하여 충돌하는 asyncio 작업을 생성할 수 있습니다.

**해결 방법**: `SequentialToolExecutor()`를 사용하여 모든 tool 호출이 순차적으로 실행되도록 보장하여, 동일한 기능을 유지하면서 비동기 작업 충돌을 제거합니다.


## 📚 추가 자료

Web Bot Auth와 AI agent의 CAPTCHA를 줄이는 방법에 대해 자세히 알아보려면 다음 리소스를 확인하세요:

### AWS 블로그 게시물
- **[Reduce CAPTCHAs for AI agents browsing the web with Web Bot Auth (Preview) in Amazon Bedrock AgentCore Browser](https://aws.amazon.com/blogs/machine-learning/reduce-captchas-for-ai-agents-browsing-the-web-with-web-bot-auth-preview-in-amazon-bedrock-agentcore-browser/)** - Web Bot Auth 구현 및 이점에 대한 종합 가이드

### 문서
- **[Cloudflare Web Bot Auth Documentation](https://developers.cloudflare.com/bots/reference/bot-verification/web-bot-auth/)** - 기술 사양 및 구현 세부사항
- **[Amazon Bedrock AgentCore Browser Documentation](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/browser-onboarding.html)** - AgentCore Browser 기능에 대한 완전한 가이드

### 관련 주제
- **[HTTP Message Signatures (RFC 9421)](https://datatracker.ietf.org/doc/rfc9421/)** - Web Bot Auth에서 사용하는 기본 암호화 표준
- **[Web Bot Auth Architecture (IETF Draft)](https://datatracker.ietf.org/doc/html/draft-meunier-web-bot-auth-architecture)** - Web Bot Auth 아키텍처에 대한 IETF 초안 사양